# Documenting the touchstone format
[Touchstone][touchstone] is a series of formats for saving s-parameters and related data for network analyzer measurements. It is an ascii based format that can have several extensions associated with it. For a number of given number of ports it may have the file extension .snp where n is the number of ports. The most common extension is s2p, but other port numbers can exist and .ts can also represent a touchstone file of unknown port number.

From the app-note on touchstone:
>"Touchstone files are ASCII text files in which frequency dependent data appears line by line, one line per data point, in increasing order of frequency. Each frequency line consists of a frequency value and one or more pairs of values for the magnitude and phase of each S-parameter at that frequency. Values are separated by one or more spaces, tabs or commands. Comments are preceded by an exclamation mark (!). Comments can appear on separate lines, or after the data on any line or lines. Extra spaces are ignored." 



[touchstone]:http://cp.literature.agilent.com/litweb/pdf/genesys200801/sim/linear_sim/sparams/touchstone_file_format.htm

## Extracting a Data Row Using Regular Expressions

for a 2 port row we would have something like:

In [6]:
import re
test_row_string='10.0e10  .1 .001 .9 180.00 .9 180.00\t.1 .001\n'
test_row_string_2=' 1.0000  0.3926  -0.1211  -0.0003  -0.0021  -0.0003  -0.0021  0.3926  -0.1211\n'
num_match=r'[+-]?(\d+(\.\d*)?|\.\d+)([eE][+-]?\d+)?'
names=["Frequency","magS11","argS11","magS21","argS21","magS12","argS12","magS22","argS22"]
row_regex="[\s|.]+"
for index,name in enumerate(names):
    if index == len(names)-1:
        row_regex=row_regex+'(?P<%s>{0})'%name
    else:
        row_regex=row_regex+'(?P<%s>{0})[\s]+'%name
row_regex=row_regex.format(num_match)
#print row_regex
row_match=re.compile(row_regex)
match=re.match(row_match,test_row_string_2)
row_data=match.groupdict()
print match.groupdict()
data_list=re.split('[\s]+',test_row_string)
print data_list

{'argS12': '-0.0021', 'argS11': '-0.1211', 'magS11': '0.3926', 'argS22': '-0.1211', 'argS21': '-0.0021', 'magS12': '-0.0003', 'Frequency': '1.0000', 'magS21': '-0.0003', 'magS22': '0.3926'}
['10.0e10', '.1', '.001', '.9', '180.00', '.9', '180.00', '.1', '.001', '']


In [2]:
float(match.groupdict()['Frequency'])

1.0

Now this matches a single row of data in the MA format. A similar formatter string can be used to match the DB and RI types.

## Extracting a comment using regular expressions

In [3]:
COMMENT_PATTERN="!(?P<Comment>.+)\n"
test_comment="! This is a comment\n"
match=re.search(COMMENT_PATTERN,test_comment)
print match.groupdict()
test_comment_2=test_row_string+"! This is also a comment\n"
match=re.search(COMMENT_PATTERN,test_comment_2)
print match.groupdict()

{'Comment': ' This is a comment'}
{'Comment': ' This is also a comment'}


## Extracting the option row values using regular expressions

In [4]:
OPTION_LINE_PATTERN="""#[\s]+(?P<Frequency_Units>\w+)[\s]+(?P<Parameter>\w+)[\s]+(?P<Format>\w+)[\s]+R[\s]+(?P<Reference_Resistance>[+-]?(\d+(\.\d*)?|\.\d+)([eE][+-]?\d+)?)"""
test_option_line="# GHZ S RI R 50.0 "
match=re.search(OPTION_LINE_PATTERN,test_option_line)
print match.groupdict()

{'Reference_Resistance': '50.0', 'Parameter': 'S', 'Frequency_Units': 'GHZ', 'Format': 'RI'}


## Converting the columns of sparameters to complex numbers


In [7]:
data_dictionary={key:float(value) for key,value in row_data.iteritems()}


In [9]:
import cmath,math
math.pi

3.141592653589793

In [10]:
clist=[data_dictionary["Frequency"],cmath.rect(data_dictionary["magS11"],(math.pi/180)*data_dictionary["argS11"])]

In [12]:
clist[1]

(0.3925991230735705-0.0008297962782850733j)

# My issues with snp formats:
1. The order of 2 ports should be S11 S12 S21 S22
2. Line comments and inline comments are mixed. 
    + Line Comments can appear in any line including data lines, header lines, lines before and lines after data
    + Inline comments are allowed at the ends of data lines but still only have ! and "\n" as delimiters
3. The matrix format seems to be formated for display and not content purposes. Wrapping the matrix after 4 S parameters is an arbitrary choice most likely driven by the 80 character line limit. This does not make sense in a world with 24 bit depth values
4. The column_names are implicitly given by an optional line
5. Data delimiter can be any combination of spaces, tabs, or command statements. This can be handled on read by a regex, but keeping the original delimiter is hard and stupid
6. There is no record or row terminator. It in practice is an endline before a frequency value
7. All types of snp file are a single data table except s2p which can be two data tables (Sparameter and Noise Parameter) end to end. This is fundementally a different data type and a second file should be generated.
8. No idenitifed method for handling switch terms